In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('../data/linkfree.csv')

In [2]:
df.head()

,type,linkfree
0,INFJ,"[['enfp', 'intj', 'moment', 'sportscenter', 't..."
1,ENTP,"[['im', 'finding', 'lack', 'post', 'alarming']..."
2,INTP,"[['good', 'one'], ['course', 'say', 'know', 't..."
3,INTJ,"[['dear', 'intp', 'enjoyed', 'conversation', '..."
4,ENTJ,"[['youre', 'fired'], ['thats', 'another', 'sil..."


In [3]:
X = df['linkfree']
y = df['type']

In [4]:
#split data into train and test
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3, random_state = 1)

# Embedding with Word2Vec

In [5]:
from gensim.models import Word2Vec

word2vec = Word2Vec(sentences=X_train)

# Convert X on NetWork Ready Data

In [6]:
def embed_sentence(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])
        
    return np.array(embedded_sentence)


In [7]:
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed


X_train = embedding(word2vec, X_train)
X_test = embedding(word2vec, X_test)

## Padding

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_train_pad = pad_sequences(X_train, dtype='float32', padding='post')
X_test_pad = pad_sequences(X_test, dtype='float32', padding='post')